### Importujemy ważne bilbioteki oraz montujemy dysk google drive




In [6]:
import os
!pip install biosignalsnotebooks

!apt-get install libmagic-dev

!pip install biosppy
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import butter, filtfilt
from numpy import ediff1d

import biosignalsnotebooks as bsnb

import bokeh
bokeh.io.output_notebook()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmagic-dev is already the newest version (1:5.41-3ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive


### Wczytujemy wszystkie pliki ekg txt do edycji
Opis plików:

* 1a, 1b, 2a, 2b, 3a, 3b -> EDA na nadgarstku
* 4a, 4b -> EDA połączone na opuszkach palca
* 5 ->EDA i EKG w czasie gry, pomiar EDA na nadgarstku
* 6 -> sam  pomiar EDA na nadgarstku
* 7a 7b, 8a, 8b -> Wykonano pomiary na 2 rozgrywkach. Pomiary wykonano na tej samej osobie, pomiar EDA wykonano na nadgarstku. W pierwszej rozgrywce (pliki "7a, 7b") badanemu nie udało się wygrać w wymaganym czasie, natomiast w drugiej rozgrywce (pliki "8a, 8b") badany wygrał.
* 9a 9b, 10a 10b -> Elektrody do pomiaru EDA przyklejone były powyżej nadgarstka od strony powierzchni dłoniowej, jedna obok drugiej.
wyrzucam 1b.txt bo jest dziwny

In [28]:
folder_path = "/content/drive/MyDrive/project-psaw/data/"

raw_ecg_files = [f for f in os.listdir(folder_path)
             if f.endswith(".txt") and os.path.isfile(os.path.join(folder_path, f))]
wristFiles = ['1a','1b','2a','2b','3a','3b']
# eda_files = ['3b.txt','6.txt']
exclude_files = ['6.txt', "5.txt", "3b.txt", "1b.txt"]
# raw_ecg_files = [x for x in raw_ecg_files if x not in eda_files]
raw_ecg_files = [x for x in raw_ecg_files if x not in exclude_files]
#3b i 6 to eda
for file in raw_ecg_files:
  print(file)

2b.txt
9b.txt
10b.txt
7b.txt
8b.txt
4b.txt


Wyświetalmy za każdym razem nasze pliki aby zobaczyć co można z nich wyciągnąć

In [37]:
def plotEcg(raw_ecg, time, fileName):
    bf = bokeh.plotting.figure(
      x_axis_label= "time[s]",
      y_axis_label= "raw_ecg[u]",
      width=        1200,
      height=       300,
      title="Dane pobrane z pliku: " + fileName
    )
    bf.line(time, raw_ecg, line_width=2)
    bokeh.plotting.show(bf)

def processData(filePath, fileName):
  rawData, header = bsnb.load(filePath, get_header=True)
  raw_ecg = rawData["CH1"]
  raw_eda = rawData['CH2']
  sr = header["sampling rate"]
  ecg_len = raw_ecg.shape[0]
  time = np.linspace(start=0, stop=ecg_len/sr, num=ecg_len)
  plotEcg(raw_ecg, time, fileName)
  plotEcg(raw_eda, time, fileName)
  return raw_ecg, sr, time, ecg_len, header

def plot_pantompkins(series1, name1, series2, name2, peaks=False):
  colors = bokeh.palettes.brewer['Paired'][3]
  bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='ECG (mV)', width=800, height=250)
  bf.line(time, series1, alpha=0.8, line_width=2, color=colors[0], legend_label=name1)
  if peaks:
    bf.circle(time, series2, alpha=0.8, size=10, color=colors[1], legend_label=name2)
  else:
    bf.line(time, series2, alpha=0.8, line_width=2, color=colors[1], legend_label=name2)
  bf.legend.click_policy="hide"
  bokeh.plotting.show(bf)

def bandpass_filter(signal, lowcut, highcut, fs, order=2):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

def returnMasked(mask, time, signal):
  signal[mask] = 0
  return signal

def convert_to_ecg(raw_signal, header):
  n = header["resolution"][0]
  vcc = 3
  g_ecg = 1000

  ecg = ((((raw_signal)/2**n)-0.5)*vcc)/g_ecg
  return ecg * 1000
#maski
#2b.txt
#9b.txt
#10b.txt
#7b.txt
#8b.txt
#5.txt
#4b.txt
masks = [
    lambda time: time <= 150,
    lambda time: False,
    lambda time: not ((time >= 10 and time <= 215) or (time >= 225 and time <= 245)),
    lambda time: False,
    lambda time: False,
    lambda time: False,
    lambda time: False,
]
mask = masks[1]
# for file, mask in raw_ecg_files, masks:
file = raw_ecg_files[0]
raw_ecg, sr, time, ecg_len, header = processData(folder_path + file, file)

Output hidden; open in https://colab.research.google.com to view.

## Jak widać musimy zamienić nasze dane z jakichś jednostek z biosignalPlus na miliVolty, żeby jakkolwiek zobaczyć co się tam dzieje, do tego użyjemu naszej funkcji `convert_to_ecg` oraz używamy `mask` aby wyrzucić potencjalny szum i dane do wyrzucenia

In [38]:
converted_ecg = convert_to_ecg(raw_ecg, header)

combinedMask = np.array([mask(t) for t in time])
converted_ecg = returnMasked(combinedMask, time, converted_ecg)

bf = bokeh.plotting.figure(
    x_axis_label= "time [s]",
    y_axis_label= "converted_ecg [mV]",
    width=1200,
    height=300
)
bf.line(time, converted_ecg, line_width=2)
bokeh.plotting.show(bf)


Output hidden; open in https://colab.research.google.com to view.

## Po utworzeniu wykresów i wyświetleniu naszych sygnałów EKG musimy wykorzystać algorytm Pan-Tompkinsa, aby znaleźć nasze załamki, czyli potrzebne informacje -> po kolei wykonujemy, filtr częstotliwości, wykonujemy pochodną sygnału, podnosimy nasz sygnał do drugiej potęgi, całkujemy sygnał i pobieramy nasze załamki

In [39]:
filtered_ecg = bandpass_filter(converted_ecg, lowcut=5, highcut=15, fs=sr)
plot_pantompkins(converted_ecg, "ECG signal", filtered_ecg, "Filtered ECG")

differentiated_ecg = ediff1d(filtered_ecg)

plot_pantompkins(filtered_ecg, "Filtered ECG", differentiated_ecg, "Differentiated ECG")
squared_ecg = filtered_ecg ** 2
plot_pantompkins(differentiated_ecg, "Differentiated ECG", squared_ecg, "Squared ECG")

window_size = int(0.150 * sr)
window = np.ones(window_size) / window_size

integrated_ecg = np.convolve(squared_ecg, window, mode='same')
plot_pantompkins(squared_ecg, "Squared ECG", integrated_ecg, "Integrated ECG")

peaks_found = np.empty(len(converted_ecg))
peaks_found[:] = np.nan

threshold = 0.19

for x in range(1, len(integrated_ecg) - 1):
    if (integrated_ecg[x - 1] < integrated_ecg[x] > integrated_ecg[x + 1]) and (converted_ecg[x] > threshold):
        peaks_found[x] = converted_ecg[x]

plot_pantompkins(converted_ecg, "ECG signal", peaks_found, "Peaks found", peaks=True)


Output hidden; open in https://colab.research.google.com to view.

# Nakładamy nasze dane z EDA oraz EKG

# Wnioski